In [1]:
import tabula
import re
import pandas as pd

In [64]:
# Change pdf name
df = tabula.read_pdf("20201126 Waltham Forest Privated Public Register November 2020.pdf"
                     , pages='all'
                     , pandas_options = {'header': None})

In [65]:
len(df) #should total the amount of pages in the document

76

In [68]:
df[2]

,0,1,2,3,4
0,29245,"Flat 1, 60 St Mary Road, Walthamstow, E17 9RE",Mr Asad Chaudhary,07/03/2018,07/03/2023
1,29246,"Flat 2, 60 St Mary Road, Walthamstow, E17 9RE",Mr Asad Chaudhary,07/03/2018,07/03/2023
2,29249,"Flat 3, 60 St Mary Road, Walthamstow, E17 9RE",Mr Asad Chaudhary,07/03/2018,07/03/2023
3,29251,"Flat 4, 60 St Mary Road, Walthamstow, E17 9RE",Mr Asad Chaudhary,07/03/2018,07/03/2023
4,29256,"15 Exeter Road, Walthamstow, E17 7QJ",Mr Asad Chaudhary,08/03/2018,08/03/2023
5,29276,"Flat D, 534 High Road Leytonstone, Leytonstone...",Mr Asad Chaudhary,22/03/2018,22/03/2023
6,29278,"Flat 1, 479a High Road Leytonstone, Leytonston...",Mr Asad Chaudhary,09/03/2018,09/03/2023
7,29279,"Flat 2, 479a High Road Leytonstone, Leytonston...",Mr Asad Chaudhary,04/02/2016,04/02/2021
8,29280,"Flat 5, 481e High Road Leytonstone, Leytonston...",Mr Asad Chaudhary,04/02/2016,04/02/2021
9,29281,"Flat 6, 479a High Road Leytonstone, Leytonston...",Mr Asad Chaudhary,09/03/2018,09/03/2023


In [45]:
class SplitFields:
    def __init__(self, field):
        self.field = field
    
    @property
    def split_list(self):
        return re.split(r'([A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2})', self.field)
    
    @property
    def property_address(self):
        return ''.join(self.split_list[:len(self.split_list)-1])
    
    @property
    def licence_holder(self):
        return self.split_list[-1]
        

In [ ]:
column_names = ['ref_no', 'property_address', 'licence_holder', 'start_date', 'end_date']

In [80]:
complete_register = pd.concat(df, ignore_index=True)

In [82]:
complete_register.head()

,0,1,2,3,4
0,Ref.No.,Property Address Licence Holder,NaN,Start Date,End Date
1,14531,"43 Chester Road, Walthamstow, E17 7HP Mr Lahri...",NaN,15/03/2018,15/03/2023
2,20502,"Flat 1, 8 Goldsmith Road, Leyton, E10 5HA Mr L...",NaN,12/05/2017,12/05/2022
3,20881,"Flat 2, 79 Park Road, Leyton, E10 7BZ Mr Lahri...",NaN,01/11/2018,01/11/2023
4,20932,"Ground Floor Flat, 23 Cornwallis Road, Waltham...",NaN,26/10/2018,26/10/2023


In [83]:
# remove first line 
complete_register.drop([0], inplace=True)

In [85]:
complete_register.reset_index(drop=True, inplace=True)

In [88]:
complete_register.head(20)

,0,1,2,3,4
0,14531,"43 Chester Road, Walthamstow, E17 7HP Mr Lahri...",NaN,15/03/2018,15/03/2023
1,20502,"Flat 1, 8 Goldsmith Road, Leyton, E10 5HA Mr L...",NaN,12/05/2017,12/05/2022
2,20881,"Flat 2, 79 Park Road, Leyton, E10 7BZ Mr Lahri...",NaN,01/11/2018,01/11/2023
3,20932,"Ground Floor Flat, 23 Cornwallis Road, Waltham...",NaN,26/10/2018,26/10/2023
4,21254,"Flat 3, 62 St James Street, Walthamstow, E17 7...",NaN,26/10/2018,26/10/2023
5,21484,"Flat 1, 165 Sinclair Road, Chingford, E4 8PP M...",NaN,22/06/2016,22/06/2021
6,22965,"24 Mayfield Road, Walthamstow, E17 5RH Mr Davi...",NaN,04/01/2016,04/01/2021
7,24877,"Ground Floor Flat, 174 Blackhorse Lane, Waltha...",NaN,05/11/2018,05/11/2023
8,25327,"Ground Floor Studio Flat, 110 Markhouse Road, ...",NaN,13/05/2019,13/05/2024
9,25335,"Ground Floor Rear Flat, 110 Markhouse Road, Wa...",NaN,18/04/2019,18/04/2024


In [87]:
# spot check
complete_register[complete_register[0]==50844]

,0,1,2,3,4
1458,50844,"75 Barrett Road, Walthamstow, E17 9ES",Mr Saleem Sheikh,01/10/2019,01/10/2024


In [ ]:
hmo_count = 0
for page in df:
    hmo_count += len(page)
print(hmo_count)

In [95]:
pd.isnull(complete_register[2][0])

True

In [92]:

if complete_register[2][0] == 'nan':
    print(1)

In [ ]:
pd.isnull(complete_register[2][0])